In [1]:
using CSV
using DataFrames
using PrettyTables
using StatsBase
include("./advgames.jl")
include("./advgames_analysis.jl")

variance_inclinations (generic function with 1 method)

In [2]:
popsize = 121
num_cooperators = Int(popsize % 2 == 0 ? popsize / 2 : (popsize + 1) / 2)
num_defectors = Int(popsize - num_cooperators)  
println("Population size: ", popsize)
println("Number of neutral agents: ", num_neutral)
println("Number of cooperators: ", num_cooperators)
println("Number of defectors: ", num_defectors)


Population size: 121


UndefVarError: UndefVarError: `num_neutral` not defined

In [3]:
function run_simulation(df, num_simulations, numrounds, params_for_model, params_filename)
    for nsim in 1:num_simulations
        adata = [pr_coop]
        # model data to record
        mdata = [
            cc, 
            cd, 
            dc, 
            dd, 
            game_type,
            avg_incl,
            pr_adv_coop,
            pr_hyp_adv_c_play_d, 
            pr_hyp_adv_d_play_c, 
            perc_cooperative,
            perc_uncooperative,
            perc_middle,
            variance_inclinations
            ]
        # params is a dictionary with the parameters to be used in the simulation
        model, parameters = initialize(; params_for_model...)            
        #pretty_table(parameters, show_subheader=false)
        # write the parameters to a txt file
        open("$(params_filename)_$(nsim).txt", "w") do io
            pretty_table(io, parameters, show_subheader=false)
        end

        _, mdf = run!(model, dummystep, model_step!, numrounds; adata, mdata)
        
        # average the mdf avg_incl
        cai = mean(mdf.avg_incl)

        # proportion of the times perc_cooperative >= 0.75
        time_on_top = sum(mdf.perc_cooperative .>= 0.75) / length(mdf.perc_cooperative) 
        time_on_bottom = sum(mdf.perc_uncooperative .>= 0.75) / length(mdf.perc_cooperative) 

        if time_on_bottom == 0
            coop_degree = Inf
        else
            coop_degree = time_on_top / time_on_bottom
        end
        
        order_degree = time_on_top + time_on_bottom

        R = model.base_payouts[:C, :C]
        S = model.base_payouts[:C, :D]
        T = model.base_payouts[:D, :C]
        P = model.base_payouts[:D, :D]
        tolerance = parameters[:tolerance]
        num_defectors = parameters[:num_defectors]
        num_cooperators = parameters[:num_cooperators]
        num_neutral = parameters[:num_neutral]
        max_positive_pressure = parameters[:max_pos_neg_pressures][1]
        max_negative_pressure = parameters[:max_pos_neg_pressures][2]
        adv_mutation_rate = parameters[:a_m_mutation_rates][1]
        move_mutation_rate = parameters[:a_m_mutation_rates][2]
        weight_of_present_for_adv = parameters[:a_m_weights_of_present][1]
        weight_of_present_for_move = parameters[:a_m_weights_of_present][2]
        adv_reassess_frequency = parameters[:a_m_reassess_frequencies][1]
        move_reassess_frequency = parameters[:a_m_reassess_frequencies][2]
        move_increment = parameters[:m_increment]
        push!(df, [cai, coop_degree, order_degree, nsim, numrounds, R, S, T, P, tolerance, num_defectors, num_cooperators, num_neutral, max_positive_pressure, max_negative_pressure, adv_mutation_rate, move_mutation_rate, weight_of_present_for_adv, weight_of_present_for_move, adv_reassess_frequency, move_reassess_frequency, move_increment])
    end
end


run_simulation (generic function with 1 method)

In [4]:
max_pressures = [4, 8, 12, 16, 24, 48]
popsizes = [16, 25, 36, 49, 64, 81, 100, 400]
numrounds = 500_000
num_simulations = 50
df = DataFrame(
    cai=Float64[],
    coop_degree=Float64[], 
    order_degree=Float64[], 
    numsim = Int[],
    numrounds=Int[],
    R=Float64[], 
    S=Float64[], 
    T=Float64[], 
    P=Float64[], 
    tolerance=Float64[], 
    num_defectors=Int[], 
    num_cooperators=Int[], 
    num_neutral=Int[], 
    max_positive_pressure=Float64[], 
    max_negative_pressure=Float64[], 
    adv_mutation_rate=Float64[], 
    move_mutation_rate=Float64[], 
    weight_of_present_for_adv=Float64[], 
    weight_of_present_for_move=Float64[], 
    adv_reassess_frequency=Float64[], 
    move_reassess_frequency=Float64[], 
    move_increment=Float64[]
)

for popsize in popsizes
    for max_pressure in max_pressures

        println("Running simulation for popsize: $popsize and max_pressure: $max_pressure")

        # divide popsize into two equal parts
        # num_neutral is what is left over if popsize is not divisible by 2
        num_cooperators = Int(popsize % 2 == 0 ? popsize / 2 : (popsize + 1) / 2)
        
        max_pos_pressure = Int(max_pressure % 2 == 0 ? max_pressure / 2 : (max_pressure + 1) / 2)
        params_for_model = Dict(
            :num_cooperators => num_cooperators, 
            :num_defectors => popsize - num_cooperators, 
            :num_neutral => 0, 
            :max_pos_pressure => max_pos_pressure,
            :max_neg_pressure => max_pressure - max_pos_pressure,
            :adv_mutation_rate => 0.2,
            :move_mutation_rate => 0.15,
            :tolerance => 0.05,
            :payouts => Dict(
                (:C, :C) => 3.0,
                (:C, :D) => 0.0,
                (:D, :C) => 4.0,
                (:D, :D) => 1.0
                )
        )
        run_simulation(df, num_simulations, numrounds, params_for_model, "./diff_pops_pressures/params_$(popsize)_$(max_pressure)")
        CSV.write("diff_pops_pressures/diff_pops_pressures.csv", df)
    end
end


Running simulation for popsize: 16 and max_pressure: 4
Running simulation for popsize: 16 and max_pressure: 8
Running simulation for popsize: 16 and max_pressure: 12
Running simulation for popsize: 16 and max_pressure: 16
Running simulation for popsize: 16 and max_pressure: 24
Running simulation for popsize: 16 and max_pressure: 48
Running simulation for popsize: 25 and max_pressure: 4
Running simulation for popsize: 25 and max_pressure: 8
Running simulation for popsize: 25 and max_pressure: 12
Running simulation for popsize: 25 and max_pressure: 16
Running simulation for popsize: 25 and max_pressure: 24
Running simulation for popsize: 25 and max_pressure: 48
Running simulation for popsize: 36 and max_pressure: 4
Running simulation for popsize: 36 and max_pressure: 8
Running simulation for popsize: 36 and max_pressure: 12
Running simulation for popsize: 36 and max_pressure: 16
Running simulation for popsize: 36 and max_pressure: 24
Running simulation for popsize: 36 and max_pressure: 4